### use knn to construct graph ###


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tqdm import tqdm

from spektral.datasets.citation import Cora
from spektral.layers.convolutional import GCSConv
from spektral.layers.ops import sp_matrix_to_sp_tensor
from spektral.layers.pooling import MinCutPool
from spektral.utils.convolution import normalized_adjacency

import scipy.sparse as sp
from scipy import sparse

def load_data():
    features = np.load('./data/cifar10-features.npy')
    
    labels = np.load('./data/cifar10-labels.npy')
   
    import faiss
    
    n, dim = features.shape[0], features.shape[1]
    index = faiss.IndexFlatIP(dim)
    index = faiss.index_cpu_to_all_gpus(index)
    index.add(features)
    distances, indices = index.search(features, 26)
    
    link = np.reshape(distances[:,1:],(-1,1))
    link = link.squeeze(1)
    topk = indices
    edges = []
    for i in range(len(topk)):

        for j in range(1,len(topk[0])):
            tmp = [i]
            tmp.append(topk[i][j]) 
            edges.append(tmp)
    edges = np.array(edges)        
    adj = sp.coo_matrix((link, (edges[:, 0], edges[:, 1])),
                        shape=(topk.shape[0], topk.shape[0]),
                        dtype=np.float32)
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
    
    
    
    return adj,features,labels

def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    mx = mx.astype(np.float32)
    return mx

def _hungarian_match(flat_preds, flat_targets, preds_k, targets_k):
    # Based on implementation from IIC
    num_samples = flat_targets.shape[0]

    assert (preds_k == targets_k)  # one to one
    num_k = preds_k
    num_correct = np.zeros((num_k, num_k))

    for c1 in range(num_k):
        for c2 in range(num_k):
            # elementwise, so each sample contributes once
            votes = int(((flat_preds == c1) * (flat_targets == c2)).sum())
            num_correct[c1, c2] = votes

    # num_correct is small
    match = linear_sum_assignment(num_samples - num_correct)
    match = np.array(list(zip(*match)))

    # return as list of tuples, out_c to gt_c
    res = []
    for out_c, gt_c in match:
        res.append((out_c, gt_c))

    return res

### load data

In [ ]:
A,X,y = load_data()

n_clusters = y.max() + 1

### clustering

In [ ]:
@tf.function
def train_step(inputs):
    with tf.GradientTape() as tape:
        _, S_pool = model(inputs, training=True)
        loss = sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    opt.apply_gradients(zip(gradients, model.trainable_variables))
    return model.losses[0], model.losses[1], S_pool

from tensorboardX import SummaryWriter
from scipy.optimize import linear_sum_assignment
from sklearn import metrics

log_dir = './result_cifar10/logdir/'

writer = SummaryWriter(log_dir=log_dir)
np.random.seed(1)
epochs = 5000  # Training iterations
lr = 5e-4  # Learning rate

################################################################################
# LOAD DATASET
################################################################################
#dataset = Cora()
#adj, x, y = dataset[0].a, dataset[0].x, dataset[0].y

a_norm = normalized_adjacency(A)
a_norm = sp_matrix_to_sp_tensor(a_norm)
F = X.shape[-1]
#y = np.argmax(y, axis=-1)
n_clusters = y.max() + 1

################################################################################
# MODEL
################################################################################
x_in = Input(shape=(F,), name="X_in",dtype='float32')
a_in = Input(shape=(None,), name="A_in", sparse=True,dtype='float32')

x_0 = GCSConv(32, activation="elu")([x_in, a_in])
x_1 = GCSConv(16, activation="elu")([x_0, a_in])
x_1, a_1, s_1 = MinCutPool(n_clusters, return_mask=True)([x_1, a_in])

model = Model([x_in, a_in], [x_1, s_1])

################################################################################
# TRAINING
################################################################################
# Setup
inputs = [X, a_norm]
opt = tf.keras.optimizers.Adam(learning_rate=lr)

# Fit model
loss_history = []
nmi_history = []
for epoch in tqdm(range(epochs)):
    outs = train_step(inputs)
    outs = [o.numpy() for o in outs]
    loss_history.append((outs[0], outs[1], (outs[0] + outs[1])))
    c = np.argmax(outs[2], axis=-1)
    #nmi_history.append(v_measure_score(y,c))
    num_elems = y.shape[0]
    match = _hungarian_match(c,y,10,10)
    reordered_preds = np.zeros(num_elems, dtype=c.dtype)
    for pred_i, target_i in match:
        reordered_preds[c == int(pred_i)] = int(target_i)

    acc = int((reordered_preds == y).sum()) / float(num_elems)
    nmi = metrics.normalized_mutual_info_score(y, c)
    ari = metrics.adjusted_rand_score(y, c)
    
    writer.add_scalar('train_loss',outs[0]+outs[1],epoch)
    writer.add_scalar('acc',acc,epoch)
    writer.add_scalar('nmi',nmi,epoch)
    writer.add_scalar('ari',nmi,epoch)
loss_history = np.array(loss_history)

################################################################################
# RESULTS
################################################################################
_, s_out = model(inputs, training=False)
s_out = np.argmax(s_out, axis=-1)
print(acc)
print(nmi)
print(ari)